You should remove the `raise` exceptions below and insert your code in their place. The cells which say `DO NOT CHANGE THE CONTENT OF THIS CELL` are there to help you, if they fail, it's probably an indication of the fact that your code is wrong. You should not change their content - if you change them to make them correspond to what your program is producing, you will still not get the marks.

If you encounter an error while running your notebook that doesn't appear to be connected to RDDs (such as missing `imp`), you should check that you've run the initialization cells since you've started your latest cluster.

Before you turn your solution in, make sure everything runs as expected. With an attached cluster, you should **Clear State and Results** (under the **Clear** dropdown menu) and then click on the **Run all** icon. This runs all cells in the notebook from new. You should only submit this notebook if all cells run.

This homework is to be completed on your own. By the act of following these instructions and handing your work in, it is deemed that you have read and understand the rules on plagiarism as written in your student handbook.

We will use the historical World cup player dataset (the second cell of this notebook downloads this for you) which is in JSON format. The first two cells set the environment up for you, including downloading the file. The initial dataframe is also created for you, so your work starts when you start exploring the data in the three ways we have seen in lectures.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import imp

try:
    imp.find_module('dbutils')
except ImportError:
    import pyspark
    sc = pyspark.SparkContext()
    from pyspark.sql import SQLContext
    spark = SQLContext(sc)

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import urllib.request

def file_exists(path):
    try:
        dbutils.fs.ls(path)
        return True
    except Exception as e:
        if 'java.io.FileNotFoundException' in str(e):
            return False
        else:
            raise

player_json = "/FileStore/all-world-cup-players.json"

try:
    imp.find_module('dbutils')
    if not file_exists(player_json) == True:
        # Download to local /tmp
        urllib.request.urlretrieve("https://github.com/jokecamp/FootballData/raw/master/World%20Cups/all-world-cup-players.json", "/tmp/all-world-cup-players.json")
        dbutils.fs.cp("file:/tmp/all-world-cup-players.json", player_json)
except ImportError:
    print("Are you running this on Databricks?")

The cell below reads the data into a dataframe named `playersDF`. If you look at the file, you'll see that it's not formed quite as Spark expects: it doesn't have a single line per json record. We therefore use the `multiline` option.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
playersDF = spark.read.option("multiline","true").json(player_json)
assert playersDF.count() == 9443, "Something has gone wrong with the reading process"

We will now explore three different ways to extract the same information from the data. 
1. Via DataFrames directly
2. Via Views
3. Via RDDs

Let's start with the DataFrames. Use DataFrame operations to create a `teamNamesFromDF` DataFrame which contains all the team names from 2014 (only). (You may want to look at the DataFrame you have read in first.) The team names should only appear once in your resulting dataframe.

In [0]:
playersDF.show()

+--------------------+-----------+-----------+-----------+--------------------+---------+------+--------+---------+----+
 Club|ClubCountry|Competition|DateOfBirth| FullName|IsCaptain|Number|Position| Team|Year|
+--------------------+-----------+-----------+-----------+--------------------+---------+------+--------+---------+----+
Club AtlÃ©tico Ta...| Argentina| World Cup| 1905-5-5| Ãngel Bossio| false| | GK|Argentina|1930|
Quilmes AtlÃ©tico...| Argentina| World Cup| 1908-10-23| Juan Botasso| false| | GK|Argentina|1930|
 Boca Junio| Argentina| World Cup| 1907-2-23| Roberto Cherro| false| | FW|Argentina|1930|
Central Norte TucumÃ| Argentina| World Cup| 1907-2-23| Alberto Chividini| false| | DF|Argentina|1930|
Club Atletico Est...| Argentina| World Cup| 1909-3-19| | false| 10| FW|Argentina|1930|
Racing Club de Av...| Argentina| World Cup| 1906-3-23| | false| | DF|Argentina|1930|
 Sportivo Barrac| Argentina| World Cup| 1902-6-20| Juan Evaristo| false| | DF|Argentina|1930|
 Boca Junio| Argentina| World Cup| 1908-12-10| Mario Evaristo| false| | FW|Argentina|1930|
Estudiantes de La...| Argentina| World Cup| 1905-10-22| Manuel Ferreira| true| | FW|Argentina|1930|
Club AtlÃ©tico Sa...| Argentina| World Cup| 1901-5-15| Luis Monti| false| | MF|Argentina|1930|
 Boca Junio| Argentina| World Cup| 1899-3-12| | false| | DF|Argentina|1930|
Club Atletico Est...| Argentina| World Cup| 1905-1-1| Rodolfo Orlandini| false| | DF|Argentina|1930|
Racing Club de Av...| Argentina| World Cup| 1903-5-24|Fernando Paternoster| false| | DF|Argentina|1930|
Racing Club de Av...| Argentina| World Cup| 1900-12-28| Natalio Perinetti| false| | FW|Argentina|1930|
Club Sportivo Bue...| Argentina| World Cup| 1908-9-13| Carlos Peucelle| false| | FW|Argentina|1930|
Club AtlÃ©tico La...| Argentina| World Cup| 1910-10-3| Edmundo Piaggio| false| | DF|Argentina|1930|
Estudiantes de La...| Argentina| World Cup| 1908-5-12| Alejandro Scopelli| false| | FW|Argentina|1930|
Club AtlÃ©tico La...| Argentina| World Cup| 1902-2-5| Carlos Spadaro| false| 11| FW|Argentina|1930|
Club AtlÃ©tico Hu...| Argentina| World Cup| 1905-1-17| | false| | FW|Argentina|1930|
 Boca Junio| Argentina| World Cup| 1909-12-21| | false| | MF|Argentina|1930|
+--------------------+-----------+-----------+-----------+--------------------+---------+------+--------+---------+----+
only showing top 20 rows

In [0]:
# Your answer should have the format

teamNamesFromDF = playersDF.select('Team').where(playersDF.Year.like('2014')).distinct()
teamNamesFromDF.show()
# raise NotImplementedError()

+-----------+
 Team|
+-----------+
Ivory Coast|
 France|
 Greece|
 Argentina|
 Ecuador|
 Chile|
 Croatia|
 Italy|
 Spain|
 Uruguay|
 Mexico|
 Honduras|
Switzerland|
 Brazil|
 Japan|
 England|
 Cameroon|
 Australia|
 Costa Rica|
 Colombia|
+-----------+
only showing top 20 rows

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
from pyspark.sql import DataFrame
assert isinstance(teamNamesFromDF, DataFrame), "Your answer should be a dataframe"
assert teamNamesFromDF.count() == 32, "Unexpected number of teams"

Now do the same via constructing a temporary view called `players` from the data (remember that you'll need to ensure that the program doesn't fail if you run it twice - i.e. if the view already exists), using a Spark sql query to extract the 2014 team names (without repeats), and naming the resulting DataFrame `teamNamesFromTable`.

In [0]:
# Your answer should have the format
# teamNamesFromTable

playersDF.createOrReplaceTempView("players")
teamNamesFromTable=spark.sql("SELECT Team FROM players WHERE Year LIKE '2014'").distinct()
teamNamesFromTable.show()

# raise NotImplementedError()

+-----------+
 Team|
+-----------+
Ivory Coast|
 France|
 Greece|
 Argentina|
 Ecuador|
 Chile|
 Croatia|
 Italy|
 Spain|
 Uruguay|
 Mexico|
 Honduras|
Switzerland|
 Brazil|
 Japan|
 England|
 Cameroon|
 Australia|
 Costa Rica|
 Colombia|
+-----------+
only showing top 20 rows

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
# Check the table was created
try:
    imp.find_module('dbutils')
    tableList = [t.name for t in spark.catalog.listTables()]
    assert "players" in tableList, "You have either not created your table or you have named it something other than players"
except:
    assert ('players' in spark.tableNames()), "You have either not created your table or you have named it something other than players"
    
assert teamNamesFromTable.count() == 32, "Unexpected number of teams"

Your third implementation should go via RDDs: i.e. you'll need to create a (Row) RDD from the DataFrame data, perform `.map`, `.filter` etc operations to obtain an RDD with the same result using RDDs. Your resulting RDD should be named `teamNamesFromRDD`.

In [0]:
# Your answer should have the format
# teamNamesFromRDD = ...

temp_RDD = playersDF.rdd
teamNamesFromRDD = temp_RDD.map(lambda row: (row.Team,row.Year)).filter(lambda line: (line[1]==2014)).map(lambda fields: fields[0]).distinct()

teamNamesFromRDD.collect()

# raise NotImplementedError()

Out[16]: ['Brazil',
 'Cameroon',
 'Croatia',
 'Mexico',
 'Australia',
 'Chile',
 'Netherlands',
 'Spain',
 'Colombia',
 'Greece',
 'Ivory Coast',
 'Japan',
 'Costa Rica',
 'England',
 'Italy',
 'Uruguay',
 'Ecuador',
 'France',
 'Honduras',
 'Switzerland',
 'Argentina',
 'Bosnia and Herzegovina',
 'Iran',
 'Nigeria',
 'Germany',
 'Ghana',
 'Portugal',
 'United States',
 'Algeria',
 'Belgium',
 'Russia',
 'South Korea']

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
from pyspark.rdd import RDD
assert isinstance(teamNamesFromRDD, RDD), "Your result should be an RDD"
assert teamNamesFromRDD.count() == 32, "Unexpected number of teams"

lineage = teamNamesFromRDD.toDebugString()
assert 'MapPartitionsRDD' in lineage.decode(), "Did you really manage to answer this question via RDDs without a map?"